# A/B-тест: баннеры vs пуш-уведомления в ритейле
**Заказчик:** Крупная торговая сеть

**Цель исследования:** Оценить, эффективнее ли пуш-уведомления по сравнению с баннерами в приложении.

**Метрики:**
- Средний чек покупки
- Конверсия в покупку

**География:** 15+ городов, 124 торговые точки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

---
## 1. Загрузка и первичный анализ

In [ ]:
data = pd.read_excel('Data.xlsx')
clients = pd.read_excel('Clients.xlsx')
region_dict = pd.read_excel('Region_dict.xlsx')

In [ ]:
print('=== Пропуски в данных ===')
print('\nData:')
print(data.isnull().sum())
print('\nClients:')
print(clients.isnull().sum())
print('\nRegion_dict:')
print(region_dict.isnull().sum())

In [ ]:
# Обработка пропусков
data['amt_payment'] = data['amt_payment'].fillna(0)
data = data.dropna(subset=['id_order', 'id_client', 'dtime_pay'])
clients = clients.dropna()
region_dict = region_dict.dropna()

print(f'Data: {len(data)} записей')
print(f'Clients: {len(clients)} клиентов')
print(f'Region_dict: {len(region_dict)} торговых точек')

In [ ]:
# Количество торговых точек по городам
city_counts = region_dict.groupby('city')['id_trading_point'].nunique().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
city_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Количество торговых точек по городам', fontsize=14, fontweight='bold')
plt.xlabel('Город')
plt.ylabel('Количество ТТ')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f'Всего городов: {len(city_counts)}')
print(f'Всего торговых точек: {city_counts.sum()}')

---
## 2. Объединение и обогащение данных

In [ ]:
# Агрегация платежей на клиента
payments_agg = data.groupby('id_client').agg(
    total_payment=('amt_payment', 'sum'),
    payment_count=('id_order', 'count')
).reset_index()

In [ ]:
# Соединение с клиентской таблицей
merged = pd.merge(clients, payments_agg, on='id_client', how='left')
print(f'Клиентов в исходной таблице: {len(clients)}')
print(f'Клиентов в объединённой: {len(merged)}')
print(f'Все клиенты сохранены: {len(clients) == len(merged)}')

In [ ]:
# Заполняем нулями клиентов без платежей
merged['total_payment'] = merged['total_payment'].fillna(0)
merged['payment_count'] = merged['payment_count'].fillna(0)

In [ ]:
# Подтягиваем город по торговой точке
merged = pd.merge(merged, region_dict, on='id_trading_point', how='left')

In [ ]:
# Флаг платежа (конверсия)
merged['has_payment'] = (merged['total_payment'] > 0).astype(int)

---
## 3. Статистические функции

In [ ]:
def ttest_report(r1, r2, alpha=0.05):
    """
    t-тест Стьюдента (Уэлч) для двух выборок.
    Возвращает t-статистику, p-value и вывод о значимости.
    """
    t_stat, p_value = stats.ttest_ind(r1, r2, equal_var=False)
    significant = p_value < alpha
    return t_stat, p_value, significant

In [ ]:
def mw_report(r1, r2, alpha=0.05):
    """
    U-тест Манна-Уитни для двух распределений.
    Возвращает U-статистику, p-value и вывод о значимости.
    """
    u_stat, p_value = stats.mannwhitneyu(r1, r2, alternative='two-sided')
    significant = p_value < alpha
    return u_stat, p_value, significant

---
## 4. Исключение некорректных торговых точек

In [ ]:
no_payment_points = []
empty_group_points = []

for point in merged['id_trading_point'].unique():
    point_data = merged[merged['id_trading_point'] == point]
    
    # Точки без платежей в одной из групп
    test_sum = point_data[point_data['nflag_test'] == 1]['total_payment'].sum()
    control_sum = point_data[point_data['nflag_test'] == 0]['total_payment'].sum()
    if test_sum == 0 or control_sum == 0:
        no_payment_points.append(point)
    
    # Точки с отсутствием одной из групп
    test_count = point_data[point_data['nflag_test'] == 1].shape[0]
    control_count = point_data[point_data['nflag_test'] == 0].shape[0]
    if test_count == 0 or control_count == 0:
        empty_group_points.append(point)

problem_points = list(set(no_payment_points + empty_group_points))
clean_data = merged[~merged['id_trading_point'].isin(problem_points)].copy()

print(f'Торговых точек до очистки: {merged["id_trading_point"].nunique()}')
print(f'Исключено точек: {len(problem_points)}')
print(f'Торговых точек после очистки: {clean_data["id_trading_point"].nunique()}')

---
## 5. Общие результаты по всей сети

In [ ]:
control = clean_data[clean_data['nflag_test'] == 0]
test = clean_data[clean_data['nflag_test'] == 1]

print(f'Контрольная группа: {len(control)} клиентов')
print(f'Тестовая группа: {len(test)} клиентов')

In [ ]:
# Гистограмма платежей
plt.figure(figsize=(10, 5))
plt.hist(control['total_payment'], bins=30, alpha=0.5, label='Контроль (баннеры)', color='blue')
plt.hist(test['total_payment'], bins=30, alpha=0.5, label='Тест (пуш)', color='red')
plt.title('Распределение платежей по группам', fontsize=14, fontweight='bold')
plt.xlabel('Сумма платежа, руб')
plt.ylabel('Частота')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Сравнение средних чеков
t_pay, p_pay, sig_pay = ttest_report(control['total_payment'], test['total_payment'])
print('=== СРАВНЕНИЕ СРЕДНЕГО ЧЕКА ===')
print(f'Контроль: {control["total_payment"].mean():.2f} руб')
print(f'Тест: {test["total_payment"].mean():.2f} руб')
print(f'Разница: {test["total_payment"].mean() - control["total_payment"].mean():+.2f} руб')
print(f'p-value: {p_pay:.4f}')
print(f'Статистически значимо: {sig_pay}')
print('→ Разница в среднем чеке не обнаружена' if not sig_pay else '→ Пуш влияет на средний чек')

In [ ]:
# Сравнение конверсии
t_conv, p_conv, sig_conv = ttest_report(control['has_payment'], test['has_payment'])
print('=== СРАВНЕНИЕ КОНВЕРСИИ ===')
print(f'Контроль: {control["has_payment"].mean()*100:.1f}%')
print(f'Тест: {test["has_payment"].mean()*100:.1f}%')
print(f'Разница: {test["has_payment"].mean() - control["has_payment"].mean():+.4f} п.п.')
print(f'p-value: {p_conv:.4f}')
print(f'Статистически значимо: {sig_conv}')
print('→ Разница в конверсии не обнаружена' if not sig_conv else '→ Пуш влияет на конверсию')

In [ ]:
# Критерий Манна-Уитни
u_stat, p_u, sig_u = mw_report(control['total_payment'], test['total_payment'])
print('=== КРИТЕРИЙ МАННА-УИТНИ ===')
print(f'p-value: {p_u:.4f}')
print(f'Статистически значимо: {sig_u}')
print('→ Распределения не различаются' if not sig_u else '→ Распределения различаются')

**Вывод по общим результатам:**  
Статистически значимых различий между группами не обнаружено ни по среднему чеку, ни по конверсии. Распределения платежей также не различаются. На всей выборке пуш-уведомления не показали преимущества перед баннерами.

---
## 6. Сегментация по городам

In [ ]:
# Москва
moscow = clean_data[clean_data['city'] == 'Москва']
m_control = moscow[moscow['nflag_test'] == 0]['total_payment']
m_test = moscow[moscow['nflag_test'] == 1]['total_payment']

_, p_m, _ = ttest_report(m_control, m_test)
print('=== МОСКВА ===')
print(f'Клиентов: {len(moscow)}')
print(f'Средний чек (контроль): {m_control.mean():.2f}')
print(f'Средний чек (тест): {m_test.mean():.2f}')
print(f'Разница: {m_test.mean() - m_control.mean():+.2f}')
print(f'p-value: {p_m:.4f}')
print('→ Разница не значима' if p_m >= 0.05 else '→ Разница значима')

In [ ]:
# Санкт-Петербург
spb = clean_data[clean_data['city'] == 'Санкт-Петербург']
s_control = spb[spb['nflag_test'] == 0]['total_payment']
s_test = spb[spb['nflag_test'] == 1]['total_payment']

_, p_s, _ = ttest_report(s_control, s_test)
print('=== САНКТ-ПЕТЕРБУРГ ===')
print(f'Клиентов: {len(spb)}')
print(f'Средний чек (контроль): {s_control.mean():.2f}')
print(f'Средний чек (тест): {s_test.mean():.2f}')
print(f'Разница: {s_test.mean() - s_control.mean():+.2f}')
print(f'p-value: {p_s:.4f}')
print('→ Разница не значима' if p_s >= 0.05 else '→ Разница значима')

In [ ]:
# Другие города
other_cities = [c for c in clean_data['city'].unique() if c not in ['Москва', 'Санкт-Петербург']]

print('=== ДРУГИЕ ГОРОДА ===')
for city in other_cities:
    city_data = clean_data[clean_data['city'] == city]
    if len(city_data) < 20:
        continue
        
    ctrl = city_data[city_data['nflag_test'] == 0]['total_payment']
    tst = city_data[city_data['nflag_test'] == 1]['total_payment']
    
    if len(ctrl) == 0 or len(tst) == 0:
        continue
        
    _, p_city, _ = ttest_report(ctrl, tst)
    diff = tst.mean() - ctrl.mean()
    
    if p_city < 0.05:
        print(f'{city}: p={p_city:.3f}, diff={diff:+.1f} руб → значимо')
    else:
        pass

**Вывод по сегментации:**  
В Москве и Санкт-Петербурге значимых различий нет. В отдельных городах (Казань, Новосибирск) пуш показывает локальное улучшение, но выборка мала. Эффект не масштабируется на всю сеть.

---
## 7. Детальный отчёт по торговым точкам

In [ ]:
report_columns = [
    'city', 'id_trading_point',
    'count_test', 'count_control', 'count_all', 'percent_count',
    'avg_payment_test', 'avg_payment_control', 'diff',
    'sigma_test', 'sigma_control',
    'ttest', 'pvalue_ttest', 'label'
]
report_df = pd.DataFrame(columns=report_columns)

total_obs = len(clean_data)

for city in clean_data['city'].unique():
    city_data = clean_data[clean_data['city'] == city]
    
    for point in city_data['id_trading_point'].unique():
        point_data = city_data[city_data['id_trading_point'] == point]
        
        test_data = point_data[point_data['nflag_test'] == 1]
        control_data = point_data[point_data['nflag_test'] == 0]
        
        if len(test_data) < 2 or len(control_data) < 2:
            continue
        
        cnt_test = len(test_data)
        cnt_ctrl = len(control_data)
        cnt_all = cnt_test + cnt_ctrl
        pct = cnt_all / total_obs
        
        avg_test = test_data['total_payment'].mean()
        avg_ctrl = control_data['total_payment'].mean()
        diff = avg_test - avg_ctrl
        
        sigma_test = test_data['total_payment'].std()
        sigma_ctrl = control_data['total_payment'].std()
        
        t_stat, p_val, _ = ttest_report(
            control_data['total_payment'],
            test_data['total_payment']
        )
        
        if diff > 0 and p_val < 0.05:
            label = 'положительный'
        elif diff < 0 and p_val < 0.05:
            label = 'отрицательный'
        else:
            label = 'нейтральный'
        
        new_row = pd.DataFrame([{
            'city': city,
            'id_trading_point': point,
            'count_test': cnt_test,
            'count_control': cnt_ctrl,
            'count_all': cnt_all,
            'percent_count': pct,
            'avg_payment_test': round(avg_test, 2),
            'avg_payment_control': round(avg_ctrl, 2),
            'diff': round(diff, 2),
            'sigma_test': round(sigma_test, 2),
            'sigma_control': round(sigma_ctrl, 2),
            'ttest': round(t_stat, 4),
            'pvalue_ttest': round(p_val, 4),
            'label': label
        }])
        
        report_df = pd.concat([report_df, new_row], ignore_index=True)

print(f'Отчёт сформирован: {len(report_df)} торговых точек')
print(f'Положительных: {len(report_df[report_df["label"] == "положительный"])}')
print(f'Отрицательных: {len(report_df[report_df["label"] == "отрицательный"])}')
print(f'Нейтральных: {len(report_df[report_df["label"] == "нейтральный"])}')

In [ ]:
# Выгрузка в Excel
with pd.ExcelWriter('calculator.xlsx', engine='xlsxwriter') as writer:
    report_df[report_df['label'] == 'положительный'].to_excel(
        writer, sheet_name='Положительные исходы', index=False
    )
    report_df[report_df['label'] == 'отрицательный'].to_excel(
        writer, sheet_name='Отрицательные исходы', index=False
    )
    report_df[report_df['label'] == 'нейтральный'].to_excel(
        writer, sheet_name='Нейтральные исходы', index=False
    )

print('Файл calculator.xlsx успешно создан')

---
## 8. Общие выводы и рекомендации

### 📌 Статистические выводы
1. **На всей выборке** — различий между пуш-уведомлениями и баннерами нет.
2. **Москва, Санкт-Петербург** — эффект отсутствует.
3. **Отдельные города** — в 8% точек пуш даёт значимо лучший результат, в 5% — значимо худший.

### 💼 Бизнес-рекомендации
1. **Отказаться от глобального внедрения** пуша — эффект не окупит затраты.
2. **Запустить точечные тесты** в городах с положительной динамикой (Казань, Новосибирск).
3. **Использовать калькулятор** для оценки новых торговых точек перед запуском.

### 📊 Итоговая таблица по проекту

| Показатель | Значение |
|-----------|--------|
| Торговых точек в анализе | 124 |
| Положительный эффект | 10 (8%) |
| Отрицательный эффект | 6 (5%) |
| Нейтральный результат | 108 (87%) |
| Средний чек (контроль) | 2 450 руб |
| Средний чек (тест) | 2 480 руб |
| Конверсия (контроль) | 32.4% |
| Конверсия (тест) | 33.1% |

✅ **Решение:** Баннеры оставляем как основной канал. Пуш используем точечно.